# PRML Final PJ Part 2
## Adversarial Training for Chinese NER(Name Entity Recognition)

### 模型和基本代码

In [1]:
%reload_ext autoreload
%autoreload 2
import torch
from datasets import load_from_disk
import random
from transformers import BertTokenizer,AutoTokenizer
from transformers import BertModel
from torchcrf import CRF

#加载字典和分词工具
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese') # Why use BertTokenizer then: list out of range?
# 调参
args = {
    'max_length':128,
    'batch_size':32,
    'epoch':3,
    'init_lr':2e-5,
    'seed':521,
    'device':'cuda:0',
}

device =  args['device'] if torch.cuda.is_available() else 'cpu'
print('device=', device)

/remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device= cuda:0


In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,split = 'train'): 
        # 人民日报NER数据集
        #names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
        #在线加载数据集
        #dataset = load_dataset(path='peoples_daily_ner', split=split)
        # 离线加载数据集
        dataset = load_from_disk(dataset_path='./peoples_daily_ner')[split]
        def f(data):
            return len(data['tokens']) <= 48 - 2
        dataset = dataset.filter(f)
        self.dataset = dataset
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        tokens = self.dataset[i]['tokens']
        labels = self.dataset[i]['ner_tags']

        return tokens, labels
    

class NERModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_bert = BertModel.from_pretrained('bert-base-chinese')
        self.pretrained_bert.to(device)
        self.classifier = torch.nn.Linear(768, 8)
        self.crf = CRF(8, batch_first=True)
        self.emissions = None
    def loss(self,input_ids, attention_mask, token_type_ids, labels):
        out = self.pretrained_bert(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids).last_hidden_state
        self.emissions = self.classifier(out)
        loss = self.crf(emissions=self.emissions, tags=labels, mask=attention_mask.byte())
        return loss
    
    def forward(self,input_ids, attention_mask, token_type_ids):
        out = self.pretrained_bert(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids).last_hidden_state
        logits = self.classifier(out)
        tags = self.crf.decode(emissions=logits,  mask=attention_mask.byte())
        return tags
    
#数据整理函数
def collate_fn(data):
    tokens = [i[0] for i in data]
    labels = [i[1] for i in data]
    data = tokenizer.batch_encode_plus(tokens,
                                         truncation=True,
                                         padding=True,
                                         return_tensors='pt',
                                         is_split_into_words=True)

    lens = data['input_ids'].shape[1]
    for i in range(len(labels)):
        labels[i] = [7] + labels[i]
        labels[i] += [7] * lens
        labels[i] = labels[i][:lens]
    
    #input_ids 就是编码后的词
    #token_type_ids 第一个句子和特殊符号的位置是0,第二个句子的位置是1
    #attention_mask pad的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)
    return input_ids, attention_mask, token_type_ids, labels

#对计算结果和label变形,并且移除pad
def reshape_and_remove_pad(outs, labels, attention_mask):
    #变形,便于计算loss
    #[b, lens(经过CRF已经不一样长)] -> [b*lens] (本身CRF会只保留有效长度)
    outs = torch.tensor(sum(outs,[]),device=device)
    #[b, lens] -> [b*lens]
    labels = labels.reshape(-1)

    #忽略对pad的计算结果
    #[b, lens] -> [b*lens - pad]
    # attention_mask: pad位置是0，非pad为1
    select = attention_mask.reshape(-1) == 1
    labels = labels[select]

    return outs, labels

#获取正确数量和总数
def get_correct_and_total_count(labels, outs):
    assert(len(labels) == len(outs))
    # print('debug')
    # print(outs)
    # print(labels)
    correct = (outs == labels).sum().item()
    total = len(labels)

    #计算除了0以外元素的正确率,因为0太多了,包括的话,正确率很容易虚高
    select = labels != 0
    outs = outs[select]
    labels = labels[select]
    correct_content = (outs == labels).sum().item()
    total_content = len(labels)

    return correct, total, correct_content, total_content

# 设置随机种子函数
def setup_seed(seed=0):
    torch.manual_seed(seed)  # 为CPU设置随机种子
    random.seed(seed)  # Python random module.
    if torch.cuda.is_available():
        # torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
        torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子

In [3]:
# 初始化随机种子
setup_seed(args['seed'])
#数据加载器
loader = torch.utils.data.DataLoader(dataset=Dataset('train'),
                                     batch_size=args['batch_size'],
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)
loader_valid = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=400,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=False)
loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=400,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                            drop_last=False)

Loading cached processed dataset at peoples_daily_ner/train/cache-88d75c6faa3faef6.arrow
Loading cached processed dataset at peoples_daily_ner/validation/cache-3a479537469f0585.arrow
Loading cached processed dataset at peoples_daily_ner/test/cache-74615302e494a10a.arrow


### Runner 类
不使用对抗训练与使用几种对抗训练方法的代码

In [4]:
class NER_Runner():
    def __init__(self,model,optimizer) -> None:
        self.model =model
        self.optimizer = optimizer
        
    def train(self,train_loader,valid_loader,num_epoch=1):
        self.model.train()
        step = 0
        best_accuracy = 0
        for epoch in range(1,num_epoch+1):
            for batch_id, (input_ids, attention_mask, token_type_ids,labels) in enumerate(train_loader):
                self.model.train()
                loss = -self.model.loss(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels)
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                
                with torch.no_grad():
                    outs = self.model.crf.decode(emissions=self.model.emissions, mask=attention_mask.byte())
                    outs,labels = reshape_and_remove_pad(outs,labels,attention_mask)
                    correct, total, correct_content, total_content = get_correct_and_total_count(labels,outs)
                    score1 = float(correct/total)
                    score2 = float(correct_content/total_content)
                    
                valid_accuracy_all,valid_accuracy = self.evaluate(valid_loader)
                if valid_accuracy >= best_accuracy:
                    best_accuracy = valid_accuracy
                    self.save_model()
                    print(f'Best performance on valid set upgraded: accuracy: {best_accuracy}')
                step += 1
                torch.cuda.empty_cache()
                if step%10 == 0:
                    print(f'[epoch]:{epoch},[step]:{step},[loss]:{loss},[score1]:{score1},[score2]:{score2}')

    @torch.no_grad()
    def evaluate(self,valid_loader):
        self.model.eval()
        correct = 0
        total = 0
        correct_content = 0
        total_content = 0
        for batch_id, (input_ids, attention_mask, token_type_ids,labels) in enumerate(valid_loader):
            outs = self.model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            outs,labels = reshape_and_remove_pad(outs,labels,attention_mask)
            correct_t, total_t, correct_content_t, total_content_t = get_correct_and_total_count(labels,outs)
            correct += correct_t
            total += total_t
            correct_content += correct_content_t
            total_content += total_content_t
            #print('debug: finish batch_id = ',batch_id)
        score1 = float(correct/total)
        score2 = float(correct_content/total_content)
        return score1,score2
        
    @torch.no_grad()
    def predict(self,test_loader):
        self.load_model()
        self.model.eval()
        correct = 0
        total = 0
        correct_content = 0
        total_content = 0
        for batch_id, (input_ids, attention_mask, token_type_ids,labels) in enumerate(test_loader):
            outs = self.model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            outs,labels = reshape_and_remove_pad(outs,labels,attention_mask)
            correct_t, total_t, correct_content_t, total_content_t = get_correct_and_total_count(labels,outs)
            correct += correct_t
            total += total_t
            correct_content += correct_content_t
            total_content += total_content_t
        score1 = float(correct/total)
        score2 = float(correct_content/total_content)
        # print(total)
        print(f'score1/score2 on test set:{score1},{score2}')
        return score1,score2
    
    def save_model(self, save_path = './modelparams/ner_bestmodel_parms.pth'):
        torch.save(self.model.state_dict(), save_path)

    def load_model(self, model_path='./modelparams/ner_bestmodel_parms.pth'):
        self.model.load_state_dict(torch.load(model_path))

class NER_Runner_FGM(NER_Runner):
    def __init__(self,model,optimizer,fgm = None) -> None:
        super(NER_Runner_FGM,self).__init__(model,optimizer)
        self.fgm = fgm
        
    def train(self,train_loader,valid_loader,num_epoch=1):
        self.model.train()
        step = 0
        best_accuracy = 0
        for epoch in range(1,num_epoch+1):
            for batch_id, (input_ids, attention_mask, token_type_ids,labels) in enumerate(train_loader):
                self.model.train()
                
                loss = -self.model.loss(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels)
                loss.backward()
                self.fgm.attack()
                loss_adv = -self.model.loss(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels)
                loss_adv.backward()
                self.fgm.restore()
                self.optimizer.step()
                self.optimizer.zero_grad()
                
                with torch.no_grad():
                    outs = self.model.crf.decode(emissions=self.model.emissions, mask=attention_mask.byte())
                    outs,labels = reshape_and_remove_pad(outs,labels,attention_mask)
                    correct, total, correct_content, total_content = get_correct_and_total_count(labels,outs)
                    score1 = float(correct/total)
                    score2 = float(correct_content/total_content)
                    
                valid_accuracy_all,valid_accuracy = self.evaluate(valid_loader)
                if valid_accuracy >= best_accuracy:
                    best_accuracy = valid_accuracy
                    self.save_model()
                    print(f'Best performance on valid set upgraded: accuracy: {best_accuracy}')
                step += 1
                torch.cuda.empty_cache()
                if step%10 == 0:
                    print(f'[epoch]:{epoch},[step]:{step},[loss]:{loss},[score1]:{score1},[score2]:{score2}')

class NER_Runner_PGD(NER_Runner):
    def __init__(self,model,optimizer,pgd = None) -> None:
        super(NER_Runner_PGD,self).__init__(model,optimizer)
        self.pgd = pgd
        
    def train(self,train_loader,valid_loader,num_epoch=1):
        self.model.train()
        step = 0
        best_accuracy = 0
        K = self.pgd.k
        for epoch in range(1,num_epoch+1):
            for batch_id, (input_ids, attention_mask, token_type_ids,labels) in enumerate(train_loader):
                self.model.train()
                loss = -self.model.loss(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels)
                loss.backward()
                self.pgd.backup_grad()
                for t in range(K):
                    self.pgd.attack(is_first_attack=(t==0))
                    if t == K-1:
                        self.pgd.restore_grad()
                    else:
                        self.optimizer.zero_grad()
                        
                    loss_adv = -self.model.loss(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels)
                    loss_adv.backward() # 前面就按公式正常迭代梯度，最后一次在最初梯度上累计一次
                    
                self.pgd.restore()
                self.optimizer.step()
                self.optimizer.zero_grad()
                with torch.no_grad():
                    outs = self.model.crf.decode(emissions=self.model.emissions, mask=attention_mask.byte())
                    outs,labels = reshape_and_remove_pad(outs,labels,attention_mask)
                    correct, total, correct_content, total_content = get_correct_and_total_count(labels,outs)
                    score1 = float(correct/total)
                    score2 = float(correct_content/total_content)
                    
                valid_accuracy_all,valid_accuracy = self.evaluate(valid_loader)
                if valid_accuracy >= best_accuracy:
                    best_accuracy = valid_accuracy
                    self.save_model()
                    print(f'Best performance on valid set upgraded: accuracy: {best_accuracy}')
                step += 1
                torch.cuda.empty_cache()
                if step%10 == 0:
                    print(f'[epoch]:{epoch},[step]:{step},[loss]:{loss},[score1]:{score1},[score2]:{score2}')

class NER_Runner_FreeLB(NER_Runner):
    def __init__(self,model,optimizer,freelb = None):
        super(NER_Runner_FreeLB,self).__init__(model,optimizer)
        self.freelb = freelb
    
    def train(self,train_loader,valid_loader,num_epoch=1):
        self.model.train()
        step = 0
        best_accuracy = 0
        K = self.freelb.k
        for epoch in range(1,num_epoch+1):
            for batch_id, (input_ids, attention_mask, token_type_ids,labels) in enumerate(train_loader):
                self.model.train()
                loss = -self.model.loss(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels)
                loss.backward()
                self.optimizer.zero_grad()
                
                for t in range(K):
                    self.freelb.backup_grad()
                    self.optimizer.zero_grad()
                    self.freelb.attack(is_first_attack=(t==0))
                    loss_adv = -self.model.loss(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels)        
                    loss_adv.backward()
                    self.freelb.backup_r_grad()
                    self.freelb.upgrade_grad()
                    self.freelb.upgrade_r_at()
                    
                self.freelb.restore()
                self.optimizer.step()
                self.optimizer.zero_grad()
                with torch.no_grad():
                    outs = self.model.crf.decode(emissions=self.model.emissions, mask=attention_mask.byte())
                    outs,labels = reshape_and_remove_pad(outs,labels,attention_mask)
                    correct, total, correct_content, total_content = get_correct_and_total_count(labels,outs)
                    score1 = float(correct/total)
                    score2 = float(correct_content/total_content)
                    
                valid_accuracy_all,valid_accuracy = self.evaluate(valid_loader)
                if valid_accuracy >= best_accuracy:
                    best_accuracy = valid_accuracy
                    self.save_model()
                    print(f'Best performance on valid set upgraded: accuracy: {best_accuracy}')
                step += 1
                torch.cuda.empty_cache()
                if step%10 == 0:
                    print(f'[epoch]:{epoch},[step]:{step},[loss]:{loss},[score1]:{score1},[score2]:{score2}')

### 训练与测试

#### 不使用对抗训练

In [5]:
from torch.optim import AdamW,Adam

model = NERModel()
model.to(device)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) # 这个真的是应该小啊！2e-5 1e-5这种才可以，5e-4直接崩盘！50%都不到
runner = NER_Runner(model,optimizer)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This 

Best performance on valid set upgraded: accuracy: 0.05914054744007942
[epoch]:1,[step]:10,[loss]:849.5384521484375,[score1]:0.8070342205323194,[score2]:0.0
Best performance on valid set upgraded: accuracy: 0.20252446461494825
Best performance on valid set upgraded: accuracy: 0.37980428308041414
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: accuracy: 0.3901574244787973
Best performance on valid set upgraded: accuracy: 0.3902992483335697
Best performance on valid set upgraded: accuracy: 0.3902992483335697
Best performance on valid set upgraded: accuracy: 0.39044107218834206
Best performance on valid set upgraded: accuracy: 0.39044107218834206
Best performance on valid set upgraded: accuracy: 0.39044107218834206
[epoch]:1,[step]:20,[loss]:382.1465148925781,[score1]:0.9182915506035283,[score2]:0.42105263157894735
Best performance on valid set upgraded: accuracy: 0.39058289604311447
Best performance on valid set upgraded: accur

In [6]:
runner.predict(test_loader=loader_test)

score1/score2 on test set:0.9905299501287883,0.9784304415654888


(0.9905299501287883, 0.9784304415654888)

#### 使用FGM

In [7]:
from transformers import AdamW
from FGM import FGM
model = NERModel()
model.to(device)
fgm = FGM(model=model)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) 
runner = NER_Runner_FGM(model,optimizer,fgm = fgm)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best performance on valid set upgraded: accuracy: 0.056304070344631967
[epoch]:1,[step]:10,[loss]:787.1118774414062,[score1]:0.8333333333333334,[score2]:0.0
Best performance on valid set upgraded: accuracy: 0.25684300099276697
Best performance on valid set upgraded: accuracy: 0.3786696922422351
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: accuracy: 0.39001560062402496
[epoch]:1,[step]:20,[loss]:317.7385559082031,[score1]:0.9311348205625606,[score2]:0.4740740740740741
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: accuracy: 0.39001560062402496
Best performance on valid set upgraded: acc

In [8]:
runner.predict(test_loader=loader_test)

score1/score2 on test set:0.9922617416561627,0.9763277487621244


(0.9922617416561627, 0.9763277487621244)

#### 使用PGD

In [9]:
from transformers import AdamW
from PGD import PGD
model = NERModel()
model.to(device)
pgd = PGD(model=model)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) 
runner = NER_Runner_PGD(model,optimizer,pgd = pgd)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best performance on valid set upgraded: accuracy: 0.08239965962274855
[epoch]:1,[step]:10,[loss]:836.404541015625,[score1]:0.7948979591836735,[score2]:0.0
Best performance on valid set upgraded: accuracy: 0.17486881293433557
Best performance on valid set upgraded: accuracy: 0.3642036590554531
Best performance on valid set upgraded: accuracy: 0.38803006665721174
Best performance on valid set upgraded: accuracy: 0.3901574244787973
Best performance on valid set upgraded: accuracy: 0.39412849241242376
Best performance on valid set upgraded: accuracy: 0.4003687420224082
Best performance on valid set upgraded: accuracy: 0.4085945255992058
[epoch]:1,[step]:20,[loss]:443.4123840332031,[score1]:0.8960674157303371,[score2]:0.367816091954023
Best performance on valid set upgraded: accuracy: 0.41823854772372715
Best performance on valid set upgraded: accuracy: 0.42646433130052475
Best performance on valid set upgraded: accuracy: 0.43213728549141966
Best performance on valid set upgraded: accuracy:

In [10]:
runner.predict(test_loader=loader_test)

score1/score2 on test set:0.9925028771852907,0.9759207759614732


(0.9925028771852907, 0.9759207759614732)

#### 使用FreeLB

In [11]:
from transformers import AdamW
from FreeLB import FreeLB
model = NERModel()
model.to(device)
freelb = FreeLB(model=model)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) 
runner = NER_Runner_FreeLB(model,optimizer,freelb = freelb)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best performance on valid set upgraded: accuracy: 0.06623174017869805
[epoch]:1,[step]:10,[loss]:581.8079833984375,[score1]:0.8646895273401297,[score2]:0.0
Best performance on valid set upgraded: accuracy: 0.07814494397957736
Best performance on valid set upgraded: accuracy: 0.1794071762870515
Best performance on valid set upgraded: accuracy: 0.19571691958587434
Best performance on valid set upgraded: accuracy: 0.22365621897603177
Best performance on valid set upgraded: accuracy: 0.3093178272585449
Best performance on valid set upgraded: accuracy: 0.36831655084385195
Best performance on valid set upgraded: accuracy: 0.3898737767692526
Best performance on valid set upgraded: accuracy: 0.3954049071053751
Best performance on valid set upgraded: accuracy: 0.3995177988937739
[epoch]:1,[step]:20,[loss]:410.32818603515625,[score1]:0.893923789907312,[score2]:0.3855421686746988
Best performance on valid set upgraded: accuracy: 0.4095872925826124
Best performance on valid set upgraded: accuracy:

In [12]:
runner.predict(test_loader=loader_test)

score1/score2 on test set:0.9923275058913794,0.9782947839652717


(0.9923275058913794, 0.9782947839652717)

#### Kill iPython Kernel

In [ ]:
import os
 
pid = os.getpid()
!kill -9 $pid